In [24]:
import cv2
import numpy as np
import keying
import projection
import imp

In [48]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

In [40]:
def project_img(img,bg,mask,key_param):
    key_mask = keying.get_mask(img,key_param)
    bgra = keying.get_bgra(img,key_mask)
    res = projection.project_to_mask(bgra,mask)
    trans_img = res[:,:,:3]
    trans_mask = np.expand_dims((res[:,:,3]).astype(np.float32)/255,-1)
    result = (trans_mask * trans_img + (1 - trans_mask) * bg).astype(np.uint8)
    return result

In [106]:
def get_key_param(img):
    key_region = keying.get_region(img)
    return keying.get_params_ycrcb(img,key_region)

## IMG Test

In [124]:
img = cv2.imread("./image/twoperson.png")
bg = cv2.imread("./image/bg.jpg")
mask = cv2.imread("./image/mask.jpg")

key_param = get_key_param(img)
res = project_img(img,bg,mask,key_param)

show_images([("i",img),("bg",bg),("m",mask),("res",res)])

## Video Test

In [165]:
green_vid_path = "./video/fight_green.mp4"
env_vid_path = "./video/fight.avi"
env_mask_path = "./video/fight_mask.avi"
outp_vid = "./video/fight_result.mov"

In [166]:
cap_green = cv2.VideoCapture(green_vid_path)
cap_vid = cv2.VideoCapture(env_vid_path)
cap_mask = cv2.VideoCapture(env_mask_path)

In [167]:
ret,img = cap_green.read()
# img = np.transpose(img,[1,0,2])
ret,bg = cap_vid.read()
ret,mask = cap_mask.read()

# img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
# bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
# mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))

In [168]:
key_param = get_key_param(img)

In [142]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap_vid.get(5)
vid = cv2.VideoWriter(outp_vid,fourcc, fps, (bg.shape[1],bg.shape[0]))

In [ ]:
import time

In [143]:
while True:
    ret1,img = cap_green.read()
    ret2,bg = cap_vid.read()
    ret3,mask = cap_mask.read()
    
    if (not(ret1 and ret2 and ret3)):
        break
        
#     img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
#     bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
#     mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))
#     img = np.transpose(img,[1,0,2])    
    
    res = project_img(img,bg,mask,key_param)
    
    vid.write(res)

#     cv2.imshow("result",res)
#     ret = cv2.waitKey(20)
#     if ret==27:
#         break
cv2.destroyAllWindows()
vid.release()

In [169]:
def test():
    while True:
        ret1,img = cap_green.read()
        ret2,bg = cap_vid.read()
        ret3,mask = cap_mask.read()

        if (not(ret1 and ret2 and ret3)):
            break

    #     img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
    #     bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
    #     mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))
    #     img = np.transpose(img,[1,0,2])    

        res = project_img(img,bg,mask,key_param)

    #     cv2.imshow("result",res)
    #     ret = cv2.waitKey(20)
    #     if ret==27:
    #         break